In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
spark = SparkSession.builder.master('spark://de-zoomcamp.europe-north1-a.c.strong-keyword-364715.internal:7077')\
                    .appName('test')\
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/02 00:24:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [6]:
zones_df = spark.read.parquet('/home/ezzaldin/data/zones')
zones_df.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [7]:
locations_df = spark.read.parquet('/home/ezzaldin/data/taxi_tripdata/all_data_part.parquet')\
                    .select('PULocationID', 'DOLocationID', 'trip_distance')

In [9]:
locations_df.show(5)

+------------+------------+-------------+
|PULocationID|DOLocationID|trip_distance|
+------------+------------+-------------+
|         130|         179|         10.1|
|          97|         225|          0.0|
|          74|          75|         1.69|
|          95|          95|         0.63|
|          41|         145|          4.9|
+------------+------------+-------------+
only showing top 5 rows



In [8]:
zones_df.createOrReplaceTempView('zones')
locations_df.createOrReplaceTempView('locations')

In [10]:
locations_zone_distance_df = spark.sql("""
SELECT l.PULocationID as pickup_location,
       z1.Zone as pickup_zone,
       l.DOLocationID as dropoff_location,
       z2.Zone as dropoff_zone,
       l.trip_distance
FROM locations as l
INNER JOIN zones AS z1 ON z1.LocationID = l.PULocationID
INNER JOIN zones AS z2 ON z2.LocationID = l.DOLocationID
""") 

In [11]:
locations_zone_distance_df.show(10)

+---------------+--------------------+----------------+--------------------+-------------+
|pickup_location|         pickup_zone|dropoff_location|        dropoff_zone|trip_distance|
+---------------+--------------------+----------------+--------------------+-------------+
|            130|             Jamaica|             179|         Old Astoria|         10.1|
|             97|         Fort Greene|             225|  Stuyvesant Heights|          0.0|
|             74|   East Harlem North|              75|   East Harlem South|         1.69|
|             95|        Forest Hills|              95|        Forest Hills|         0.63|
|             41|      Central Harlem|             145|Long Island City/...|          4.9|
|            129|     Jackson Heights|             193|Queensbridge/Rave...|         2.83|
|             42|Central Harlem North|             238|Upper West Side N...|         2.54|
|             95|        Forest Hills|             135|   Kew Gardens Hills|         2.03|

In [14]:
locations_zone_distance_df.repartition(8)\
                          .write.parquet('/home/ezzaldin/data/taxi_tripdata/zones_trip_distance_loc_based.parquet')